<a href="https://colab.research.google.com/github/kamrulkonok/UPC_ML_Project/blob/Konok/merge_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import os
import pandas as pd
import plotly.graph_objects as go

In [ ]:
zip_file_path = '/content/airbnb_data.zip'

extracted_folder_path = '/content/airbnb_data/'

if not os.path.exists(extracted_folder_path):
    os.makedirs(extracted_folder_path)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

file_names = os.listdir(extracted_folder_path)
merged_df = pd.DataFrame()

In [ ]:
for file_name in file_names:
    if file_name.endswith('.csv'):
        file_path = os.path.join(extracted_folder_path, file_name)
        base_name = os.path.splitext(file_name)[0]
        parts = base_name.split('_')
        city = parts[0]
        day_type = parts[1]
        df = pd.read_csv(file_path)

        df['city'] = city
        df['day_type'] = day_type

        merged_df = pd.concat([merged_df, df], ignore_index=True)

In [ ]:
merged_df['day_type'] = pd.Categorical(merged_df['day_type'], categories=['weekdays', 'weekends'], ordered=True)
merged_df = merged_df.sort_values(by=['city', 'day_type'], ignore_index=True)
merged_df = merged_df.reset_index(drop=True)

In [ ]:
output_csv_path = '/content/airbnb_data.csv'
merged_df.to_csv(output_csv_path, index=False)

print("Merged CSV file created at:", output_csv_path)

Merged CSV file created at: /content/airbnb_data.csv


In [ ]:
df = pd.read_csv('/content/airbnb_data.csv')
df.head()

,Unnamed: 0,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,...,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat,city,day_type
0,0,194.033698,Private room,False,True,2.0,False,1,0,10.0,...,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772,amsterdam,weekdays
1,1,344.245776,Private room,False,True,4.0,False,0,0,8.0,...,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432,amsterdam,weekdays
2,2,264.101422,Private room,False,True,2.0,False,0,1,9.0,...,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103,amsterdam,weekdays
3,3,433.529398,Private room,False,True,4.0,False,0,1,9.0,...,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663,amsterdam,weekdays
4,4,485.552926,Private room,False,True,2.0,True,0,0,10.0,...,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508,amsterdam,weekdays
